In [ ]:
from manim import *
import numpy as np

config.media_width = "75%"
config.verbosity = "WARNING"

In [ ]:
#l_{p} norm
p = 2

# delta
d = 0.1

# weight vectors
weight_vecs = {
    "chip 1": {
        "w_{1}^{1}": np.array([1, 1]),
        "w_{1}^{2}": np.array([1, 1])
    },
    "chip 2": {
        "w_{2}^{1}": np.array([1, 1]),
        "w_{2}^{2}": np.array([1, 1])
    }
}

# drift vectors
drift_vecs = {
    "chip 1": {
        "z_{1}^{1}": np.array([1, 1]),
        "z_{1}^{2}": np.array([1, 1])
    },
    "chip 2": {
        "z_{2}^{1}": np.array([1, 1]),
        "z_{2}^{2}": np.array([1, 1])
    }
}